# library import

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
# import xgboost
import xgboost
from xgboost import XGBRegressor
# , plot_tree, plot_importance

In [2]:
data_df = pd.read_csv('train.csv')
test_df  = pd.read_csv('test.csv')
sub_df   = pd.read_csv('sample_submission.csv')
train_df

NameError: name 'train_df' is not defined

# 결측값 확인

## 데이터 칼럼/의미 확인

In [ ]:
data_df.columns

### 의미확인

>date: 날짜
precipitation: 강수량(mm)

> temp_mean: 평균 기온(℃)
temp_highest: 최고 기온(℃)
temp_lowest: 최저 기온(℃)

> PM10: 미세먼지(㎍/㎥)
PM2.5: 초미세먼지(㎍/㎥)

> humidity: 습도(%rh)

> sunshine_sum: 일조합
sunshine_rate: 일조율

> wind_mean: 평균 풍속(m/s)
wind_max: 최대 풍속(m/s)

> rentals : 따릉이 대여량

In [ ]:
data_df.info()

#### data_df['date'] : object -> datetime 으로 변경

In [ ]:
from datetime import datetime
import time
data_df['date'] = pd.to_datetime(data_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])
data_df.head()

### 결측치 처리는 어떻게 할 것인지에 대해서 생각

In [ ]:
data_df.isnull().sum()

In [ ]:
test_df['date'][0].year

In [ ]:
# date 에서 year 만 따로 뽑았음
data_df['date_year'] = 0
for i in range(len(data_df['date'])):
    data_df['date_year'][i] = data_df['date'][i].year
# print(data_df['date_year'])

# date 에서 year 를 제외한 Month, Day 만 넣었음.
data_df['date_month_day'] = 0
for i in range(len(data_df['date'])):
    data_df['date_month_day'][i] = data_df['date'][i].strftime('%m-%d')
# data_df['date_month_day']

# date 에서 year 만 따로 뽑았음
test_df['date_year'] = 0
for i in range(len(test_df['date'])):
    test_df['date_year'][i] = test_df['date'][i].year
# print(data_df['date_year'])

# date 에서 year 를 제외한 Month, Day 만 넣었음.
test_df['date_month_day'] = 0
for i in range(len(test_df['date'])):
    test_df['date_month_day'][i] = test_df['date'][i].strftime('%m-%d')
# data_df['date_month_day']

In [ ]:
lst = list()
lst = [i*15 for i in range(8)]
print(lst)

In [ ]:
plt.figure(figsize = (20,12))
sns.lineplot(x = 'date_month_day', y = 'rental', data = data_df, hue = 'date_year')
lst = list()
lst = [i*15 for i in range(31)]
plt.xticks(lst)
plt.show()

plt.figure(figsize = (20,12))
sns.lineplot(x = 'date', y = 'rental', data = data_df, hue = 'date_year')
plt.show()

In [ ]:
data_df['rental'].groupby(data_df.date_year).agg('median')

In [ ]:
plt.figure(figsize = (16,10))
sns.heatmap(data_df.corr(),annot = True)
plt.show()

In [ ]:
sns.pairplot(data_df, hue = 'date_year', markers = ['o','s','d'])
plt.show()

In [ ]:
data_df = data_df.drop('date_month_day', axis = 1)
test_df = test_df.drop('date_month_day', axis = 1)

In [ ]:
# 불쾌지수 공식의 활용
def get_discomfort(humididity, temp_highest, temp_lowest):
    # 전체적인 탑승의 경향성을 반영하기 위해 출퇴근 시간의 사용량이 많음에도 불구하고, 평균온도로 고려합니다.
    temp = (temp_lowest + temp_highest)/2
    humididity = humididity / 100
    
    discomfort = 1.8 * temp - 0.558 * (1 - humididity) * (1.8*temp - 26) + 32
    return discomfort

data_df['discomfort'] = [0]*len(data_df)
for i in range(len(data_df)):
    data_df.discomfort[i] = get_discomfort(data_df.humidity[i],
                                          data_df.temp_highest[i],
                                          data_df.temp_lowest[i])
    
test_df['discomfort'] = [0]*len(test_df)
for i in range(len(test_df)):
    test_df.discomfort[i] = get_discomfort(test_df.humidity[i],
                                          test_df.temp_highest[i],
                                          test_df.temp_lowest[i])

In [ ]:
# 악조건에 대한 
data_df['cold_measure'] = data_df['temp_lowest']/data_df['wind_max']
test_df['cold_measure'] = test_df['temp_lowest']/test_df['wind_max']
# 체감온도
data_df['feel'] = 13.12 + 0.6215*(data_df['temp_highest']+data_df['temp_lowest'])//2-11.37*data_df['wind_mean']**(0.16) + 0.4*(data_df['temp_highest']+data_df['temp_lowest'])//2*data_df['wind_mean']**(0.16)
test_df['feel'] = 13.12 + 0.6215*(test_df['temp_highest']+test_df['temp_lowest'])//2-11.37*test_df['wind_mean']**(0.16) + 0.4*(test_df['temp_highest']+test_df['temp_lowest'])//2*test_df['wind_mean']**(0.16)
# 일교차
data_df['temp_diff'] = data_df['temp_highest'] - data_df['temp_lowest']
test_df['temp_diff'] = test_df['temp_highest'] - test_df['temp_lowest']

In [ ]:
data_df = data_df.drop('humidity', axis =1)
test_df = test_df.drop('humidity', axis =1)

In [ ]:
data_df

In [ ]:
def get_result(pred):
    sub_df['rental'] = pred
    sub_df.to_csv('result.csv', index = False, encoding = 'UTF-8')
def get_nmae(pred,y):
    nmae = np.mean(abs(pred-y)/y)
    return nmae

In [ ]:
x = data_df.drop('rental', axis = 1)
y = data_df['rental']

In [ ]:
pip install sklearn

In [ ]:
pip install xgboost

In [ ]:
XGBRegressor_model = XGBRegressor(max_depth =4, n_estimators =100)
XGBRegressor_model.fit(x,y)
pred = XGBRegressor_model(test_df)

x_pred = XGBRegressor_model.predict()
get_nmae(x_pred,y)

# 데이터 plot 진행